<a href="https://colab.research.google.com/github/hassanSattariNia/FederatedLearning/blob/main/FL-Albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch peft scikit-learn evaluate

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split
import torch
import evaluate

In [3]:
class Client:
    def __init__(self, model, dataset,eval_dataset):
        self.model = model
        self.dataset = dataset
        self.eval_dataset = eval_dataset
        self.trainer = None

    # Function to compute accuracy
    def compute_metrics(self, eval_pred):
        accuracy_metric = evaluate.load("accuracy")
        logits, labels = eval_pred
        predictions = logits.argmax(axis=-1)
        accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
        return {"accuracy": accuracy["accuracy"]}

    def train(self):
        training_args = TrainingArguments(
        output_dir="./results",  # Directory to save checkpoints and logs
        evaluation_strategy="epoch",
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=3,
        weight_decay=0.01,
        )
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.dataset,
            eval_dataset = self.eval_dataset,
            tokenizer=tokenizer,
            compute_metrics=self.compute_metrics
        )
        self.trainer.train()

    def get_model_parameters(self):
        return self.model.state_dict()

    def update_model_parameters(self, new_params):
        self.model.load_state_dict(new_params)


In [4]:
class Server:
    def __init__(self, model):
        self.model = model

    def aggregate_parameters(self, client_parameters):
        aggregated_params = {}
        for key in client_parameters[0].keys():
            aggregated_params[key] = sum(client_param[key] for client_param in client_parameters) / len(client_parameters)
        return aggregated_params

    def update_model_parameters(self, new_params):
        self.model.load_state_dict(new_params)

    def get_model_parameters(self):
        return self.model.state_dict()

In [5]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
)

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This be

In [7]:
model = get_peft_model(model, lora_config)

#loading dataset
dataset = load_dataset("glue", "mrpc")


In [8]:
# Function to compute accuracy
def compute_metrics(eval_pred):
    accuracy_metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy["accuracy"]}

In [9]:
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length")
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [10]:
# Get the total length of the training dataset
dataset_length = len(tokenized_dataset['train'])

# Split the dataset into two equal parts
split_point = dataset_length // 2

# First part of the train dataset
train_dataset1 = tokenized_dataset['train'].select(range(split_point))

# Second part of the train dataset
train_dataset2 = tokenized_dataset['train'].select(range(split_point, dataset_length))


In [13]:
def federated_learning(num_rounds=1):
    model1 = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
    model2 = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
    model1 = get_peft_model(model1, lora_config)
    model2 = get_peft_model(model2, lora_config)

    client1 = Client(model1, train_dataset1.select(range(1000)),tokenized_dataset['validation'].select(range(100)))
    client2 = Client(model2, train_dataset2.select(range(1000)),tokenized_dataset['validation'].select(range(100)))

    server = Server(model1)


    for round in range(num_rounds):
        print(f"Round {round + 1}")

        client1.train()
        client2.train()

        client_params = [client1.get_model_parameters(), client2.get_model_parameters()]

        aggregated_params = server.aggregate_parameters(client_params)

        server.update_model_parameters(aggregated_params)
        client1.update_model_parameters(aggregated_params)
        client2.update_model_parameters(aggregated_params)

    final_model = server.model
    trainer = Trainer(
        model=final_model,
        eval_dataset=tokenized_dataset['validation'],
        tokenizer=tokenizer,
        compute_metrics = compute_metrics
    )
    results = trainer.evaluate()
    print("Final Evaluation Results:", results)

In [ ]:
federated_learning()

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Round 1


Epoch,Training Loss,Validation Loss
1,No log,No log
2,0.616500,No log
3,0.616500,No log


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
